In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier


## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Removing the observations with N/As
heart = heart.dropna()

In [8]:
## Defining the input and target variables
X = heart[['male','age', 'education','currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes','totChol', 'sysBP', 'diaBP','BMI','heartRate', 'glucose']]
Y = heart['TenYearCHD']

for i in range(0, 100):
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

    ## Building the random forest model
    RF_md1 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

    ## Predicting on test
    RF_pred1 = RF_md1.predict_proba(X_test)[:,1]

    ## Extracting the feature importances
    importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': RF_md1.feature_importances_})
    importances = importances.sort_values(by = 'Importance', ascending = False)
    importances

The recall score for the first random forest model is 0.87
The recall score for the first random forest model is 0.8823529411764706
The recall score for the first random forest model is 0.8134328358208955
The recall score for the first random forest model is 0.8244274809160306
The recall score for the first random forest model is 0.8739495798319328
The recall score for the first random forest model is 0.9
The recall score for the first random forest model is 0.8623853211009175
The recall score for the first random forest model is 0.9349593495934959
The recall score for the first random forest model is 0.9152542372881356
The recall score for the first random forest model is 0.8392857142857143
The recall score for the first random forest model is 0.864406779661017


KeyboardInterrupt: 